In [1]:
import pandas as pd
from gensim.summarization.summarizer import summarize

In [2]:
news_df = pd.read_csv('../data/news_data.csv')

summarization.summarizer – TextRank Summariser
- https://radimrehurek.com/gensim/summarization/summariser.html
- https://anpigon.github.io/blog/dclick/@anpigon/-textrank-summariser-1540351206980/
- https://soyoung-new-challenge.tistory.com/44

+ text ( str ) – input 텍스트
+ ratio ( float , optional ) – 요약을 위해 선택할 원본 텍스트의 문장 수 비율을 결정하는 0과 1 사이의 숫자.
+ word_count ( int 또는 None , 선택 사항 ) – 출력에 포함될 단어 수를 결정. 두 매개 변수가 모두 제공되면 비율이 무시.
+ split ( bool , optional ) – True이면 문장 목록이 반환. 그렇지 않으면 결합 된 문자열이 반환

In [3]:
print(summarize(news_df['contents'][0][:-50], word_count=100, ratio=0.1))

김홍걸, 당적 상실해 무소속으로…野, 의원직 사퇴요구하며 제명결정 비판
감찰단 최기상 단장은 김 의원에 대한 공직선거법 위반 및 재산 허위 신고 의혹 등에 대한 조사를 개시했으나 김 의원이 이에 대해 성실히 협조하지 않음에 따라 이낙연 대표에게 김 의원에 대한 제명을 요청했다.
최 수석대변인은 "감찰단이 여러 가지 소명이나 본인 주장을 들어보려고 했으나 성실히 응할 의사가 없다는 것을 확인했다"면서 "이 대표는 최기상 단장의 보고를 받고 즉시 긴급 최고위원회의를 소집하게 됐다"고 말했다.
정의당 조혜민 대변인도 논평에서 "의원직이 유지되는 만큼 김 의원이 마땅한 책임을 지는 결과라고 할 수 없다"며 "김 의원은 추한 모습으로 부친의 명예에 누를 끼치지 말고 의원직에서 스스로 물러나야 한다"고 말했다.


In [4]:
news_df['summarisation'][0]

'더불어민주당은 18일 재산 신고 누락 의혹이 제기된 김대중 전 대통령의 삼남 김홍걸 의원을 제명했다.\n\n감찰단 최기상 단장은 김 의원에 대한 공직선거법 위반 및 재산 허위 신고 의혹 등에 대한 조사를 개시했으나 김 의원이 이에 대해 성실히 협조하지 않음에 따라 이낙연 대표에게 김 의원에 대한 제명을 요청했다.\n\n최 수석대변인은 "감찰단이 여러 가지 소명이나 본인 주장을 들어보려고 했으나 성실히 응할 의사가 없다는 것을 확인했다"면서 "이 대표는 최기상 단장의 보고를 받고 즉시 긴급 최고위원회의를 소집하게 됐다"고 말했다.'

In [5]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

TextRank를 이용한 문서요약

- https://excelsior-cjh.tistory.com/93 (Twitter -> Okt)

In [6]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.Okt = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
                          ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", 
                          "에", "의", "가",]
    
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
    
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
            
        return sentences
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
                
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.Okt.nouns(str(sentence))
                                       if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns

<>:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-6-d15f980399f9>:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [7]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [8]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [9]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
            
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
                    
        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
            
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

In [10]:
url = 'http://v.media.daum.net/v/20170611192209012?rcmd=r'
textrank = TextRank(url)
for row in textrank.summarize(3):
    print(row)
    print()
print('keywords :',textrank.keywords())

◇ 연 봉 5000만원 대가 84% 로 카드 공제 가장 많이 받아 신용카드 소득 공제를 받으려면 총소득의 4분의 1 이상을 카드로 써야 한다.

2015년 기준으로 근로 소득세를 내는 사람 중에서 카드 소득 공제를 받는 비율이 가장 많은 소득 구간은 연봉 5000만 ~6000 만원이었다.

연봉 4000만원 대는 81%, 3000만원 대는 73%, 2000만원 대는 55% 만 카드 소득 공제를 받은 것으로 나타났다.

keywords : ['소득', '공제', '카드', '사람', '사용', '한도', '경우', '신용카드', '문턱', '설명']


In [11]:
textrank = TextRank(news_df['contents'][0])
for row in textrank.summarize(3):
    print(row)
    print()
print('keywords :',textrank.keywords())

이 낙 연, 윤리 감찰단 비상징계 요청에 긴급 최고 위 소집해 결단 김 홍 걸, 당적 상실해 무소속으로… 野, 의원직 사퇴요구하며 제명 결정 비판 질의하는 김 홍 걸 의원 [ 연합 뉴스 자료사진] ( 서울= 연합 뉴스) 김 동호 이동 환 홍규 빈 기자 = 더 불어 민주당은 18일 재산 신고 누락 의혹이 제기된 김대중 전 대통령의 삼남 김 홍 걸 의원을 제명했다.

감찰 단 최 기상 단장은 김 의원에 대한 공직 선거법 위반 및 재산 허위 신고 의혹 등에 대한 조사를 개시했으나 김 의원이 이에 대해 성실히 협조하지 않음에 따라 이 낙 연 대표에게 김 의원에 대한 제명을 요청했다.

최 의원은 브리핑 후 기자들과 만 나 ' 당에서 탈당을 요청한 적이 없느냐

keywords : ['의원', '제명', '감찰', '최고', '비상', '대변인', '뉴스', '연합', '신고', '징계']
